In [30]:
!pip install rhino3dm


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
# Non Layers Version

import numpy as np
import rhino3dm

# Function to extract geometry objects from a Rhino3dm model.
def extract_geometry(model):
    """
    Extracts geometry objects from a Rhino3dm model.
    Returns a list of geometry objects (e.g., surfaces, Breps, Extrusions).
    """
    geometry = []
    for obj in model.Objects:
        geom = obj.Geometry
        if geom:
            geometry.append(geom)
    return geometry

# Function to compute the normal vector for a given surface.
def get_normal_vector(surface):
    """
    Computes the normal vector for a given surface.
    For Extrusion objects, converts them to a Brep (calling ToBrep with a boolean argument)
    and uses the normal of the first face.
    For Brep objects, uses the normal of the first face.
    If the object has a NormalAt method, it will be used as a fallback.
    """
    try:
        type_name = type(surface).__name__
        if type_name == "Extrusion":
            # Convert the Extrusion to a Brep (providing False for splitKinkyFaces)
            brep = surface.ToBrep(False)
            if brep and len(brep.Faces) > 0:
                face = brep.Faces[0]
                normal = face.NormalAt(0.5, 0.5)
                norm_val = np.linalg.norm([normal.X, normal.Y, normal.Z])
                if norm_val != 0:
                    return np.array([normal.X, normal.Y, normal.Z]) / norm_val
        elif type_name == "Brep":
            if len(surface.Faces) > 0:
                face = surface.Faces[0]
                normal = face.NormalAt(0.5, 0.5)
                norm_val = np.linalg.norm([normal.X, normal.Y, normal.Z])
                if norm_val != 0:
                    return np.array([normal.X, normal.Y, normal.Z]) / norm_val
        # Fallback: if the surface has a NormalAt method directly.
        if hasattr(surface, "NormalAt"):
            normal = surface.NormalAt(0.5, 0.5)
            norm_val = np.linalg.norm([normal.X, normal.Y, normal.Z])
            if norm_val != 0:
                return np.array([normal.X, normal.Y, normal.Z]) / norm_val
    except Exception as e:
        print("Error computing normal for surface:", e)
    # Fallback normal (horizontal)
    return np.array([0, 0, 1])

# Function to classify geometry solely based on the computed normals.
def classify_geometry(geometry):
    classified = {"walls": [], "floors": [], "ceilings": [], "columns": [], "beams": []}
    
    for geom in geometry:
        normal = get_normal_vector(geom)
        # If the normal is nearly vertical (i.e., [0, 0, ±1]), classify as a floor.
        if np.allclose(np.abs(normal), [0, 0, 1], atol=0.2):
            classified["floors"].append(geom)
        # If the normal is nearly horizontal (i.e., [±1, 0, 0] or [0, ±1, 0]), classify as a wall.
        elif (np.allclose(np.abs(normal), [1, 0, 0], atol=0.2) or 
              np.allclose(np.abs(normal), [0, 1, 0], atol=0.2)):
            classified["walls"].append(geom)
        else:
            # Fallback: if the normal doesn't clearly indicate either, treat as floor.
            classified["floors"].append(geom)
        
        # Placeholders for additional classification for columns and beams.
        if hasattr(geom, 'IsCylinder') and geom.IsCylinder:
            classified["columns"].append(geom)
        if hasattr(geom, 'IsBeam') and geom.IsBeam:
            classified["beams"].append(geom)
            
    return classified

# Example usage for Stage 1:
file_path = "basicformMETERS.3dm"  # Replace with the actual path to your Rhino file.
try:
    model = rhino3dm.File3dm.Read(file_path)
    geometry = extract_geometry(model)
    classified_elements = classify_geometry(geometry)
    print("Geometry Classification Results:")
    for key, items in classified_elements.items():
        print(f"  {key.capitalize()}: {len(items)} element(s)")
except Exception as e:
    print("Error loading or processing the file:", e)


Geometry Classification Results:
  Walls: 20 element(s)
  Floors: 14 element(s)
  Ceilings: 0 element(s)
  Columns: 0 element(s)
  Beams: 0 element(s)


In [28]:
#normal_surface = get_normal_vector(surface)
for key,value in extract_geometry(model).items():
    print(key, value)

AttributeError: 'list' object has no attribute 'items'

In [6]:
print(dir(classified_elements))
print(help(classified_elements))

['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']
Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=

In [7]:
print("Walls: ", classified_elements["walls"])

Walls:  [<rhino3dm._rhino3dm.Brep object at 0x0000024DBDB975B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB96C70>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB94170>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB6E470>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB6E8B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB9AFB0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB9A0F0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB6F430>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBD5EF0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBD52B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBD45B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBD58B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBB84F0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBB9270>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBB8630>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBB9D70>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBEAA70>, <rhino3dm._rhino3dm.Brep object at 0x00

In [12]:
print(classified_elements.keys())

dict_keys(['walls', 'floors', 'ceilings', 'columns', 'beams'])


In [16]:
print(classified_elements.keys().values())

AttributeError: 'dict_keys' object has no attribute 'values'

In [17]:
for key,value in classified_elements.items():
    print(key, value)

walls [<rhino3dm._rhino3dm.Brep object at 0x0000024DBDB975B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB96C70>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB94170>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB6E470>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB6E8B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB9AFB0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB9A0F0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDB6F430>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBD5EF0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBD52B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBD45B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBD58B0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBB84F0>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBB9270>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBB8630>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBB9D70>, <rhino3dm._rhino3dm.Brep object at 0x0000024DBDBEAA70>, <rhino3dm._rhino3dm.Brep object at 0x0000